In [1]:
import requests
import requests_cache
from matplotlib import pyplot as plt
import lxml.html
import pandas as pd
import re
import json 
import lxml.html as lx
import urllib
from urllib2 import Request, urlopen, HTTPError
from urlparse import urlunparse, urlparse, urljoin
import numpy 
import plotly.plotly as py
import plotly.figure_factory as ff
import math
from bs4 import BeautifulSoup
import plotly.graph_objs as go 
import plotly
import plotly.tools as tls
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Scatter, Figure, Layout
init_notebook_mode(connected=True)
import cufflinks as cf
import os.path
from datetime import datetime
import pandas_datareader as web
from plotly.graph_objs import *
import ast
import HTMLParser
import string
import folium
import json
from branca.utilities import _locations_mirror
from folium.features import *
from folium.features import _locations_tolist

In [2]:
##Web craper to find the list of counties in California as county_name dataframe
def fetch_page(url):
    response = requests.get(url)
    response.raise_for_status()
    return response

url="https://www.zillow.com/browse/homes/ca/"
requests_cache.install_cache("cache")
page = fetch_page(url)
html = lx.fromstring(page.text)
county_html = html.xpath('//main/div/div/ul/li/a')
county = [i.text_content() for i in county_html]
county_name = pd.DataFrame({"RegionName":[" ".join(k) for k in [i.split(" ")[:-2] for i in county]]})

In [3]:
##Import Raw Housing Data(10 csv file) and extract the data only for counties in CA


def dataframe(i = 0):
    
    """Read datasets
    Please change the file path to fit to your computer
    Input:the name of dataset //dataname   
    """
    
    file_path = "/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/Housing_Data"
    file_name = ["County_MedianSoldPrice_AllHomes.csv","County_MedianListingPrice_AllHomes.csv","County_MedianListingPrice_1Bedroom.csv",
                 "County_MedianListingPrice_2Bedroom.csv","County_MedianListingPrice_3Bedroom.csv","County_MedianListingPrice_4Bedroom.csv",
                 "County_MedianListingPrice_5BedroomOrMore.csv","County_MedianListingPrice_CondoCoop.csv","County_MedianListingPrice_DuplexTriplex.csv",
                 "County_MedianListingPrice_Sfr.csv"]
    dataframe = pd.read_csv(os.path.join(file_path,file_name[i]),header=0)
    CA_dataframe = dataframe[dataframe["State"]== "CA"]
    CA_dataframe = CA_dataframe.reset_index(drop=True)
    return(CA_dataframe)

###name of eachdata
dataname=["sold","list","one","two","thre","four","five","cond","trip","sfr"]

###save all of dataset to CA dict.
CA = {i: dataframe(dataname.index(i)) for i in dataname}

In [4]:
#All Home Sold time-series data 1996-2016 and save as CA_sold_t
CA["sold"] = pd.merge(pd.DataFrame(county_name),CA["sold"], how='left', on=['RegionName'])
CA_sold_t = CA["sold"].transpose()
CA_sold_t = CA_sold_t.rename(columns=lambda i: CA["sold"]["RegionName"][i])[7:]
date = pd.date_range('1996-04', '2016-07', freq='M')
CA_sold_t['date'] = date

#sold price time-series data
CA_sold_t = CA_sold_t.set_index('date')
CA_sold_t.head()

,Los Angeles,Orange,San Diego,Riverside,San Bernardino,Santa Clara,Alameda,Sacramento,Contra Costa,Fresno,...,Glenn,Plumas,Colusa,Mariposa,Inyo,Trinity,Mono,Modoc,Sierra,Alpine
date,,,,,,,,,,,,,,,,,,,,,
1996-04-30,162225,191525,158225,114550,100250,234325,196275,120425,192750,94850,...,67000,NaN,NaN,NaN,121471,53000,77500,NaN,NaN,NaN
1996-05-31,162925,197550,158125,112500,101700,238575,196900,121675,203000,95475,...,65823.5,NaN,97230.8,115000,NaN,NaN,78970.6,NaN,NaN,NaN
1996-06-30,164500,197875,158925,115000,103450,238875,202225,122175,209050,95475,...,NaN,NaN,NaN,NaN,119077,51950,NaN,NaN,NaN,NaN
1996-07-31,164550,198050,158700,113975,101850,241875,203700,122175,207150,96550,...,NaN,NaN,NaN,94230.8,112882,NaN,81538.5,NaN,NaN,NaN
1996-08-31,163300,199300,157300,113225,101650,241200,202250,121075,202500,96625,...,NaN,80000,NaN,NaN,NaN,82300,98470.6,NaN,NaN,NaN


In [5]:
#Different type of house average yr 2015/2016
def aver(i="one"):
    data = CA[i]
    CA_2015 = data.iloc[:,[k.__contains__("2015") for k in data.keys()]]
    CA_2016 = data.iloc[:,[k.__contains__("2016") for k in data.keys()]]
    CA_mean = pd.DataFrame({i+"_15":CA_2015.mean(1),i+"_16":CA_2016.mean(1)})
    CA_ave_name = pd.DataFrame({"RegionName":data["RegionName"]})
    CA_ave = pd.concat([CA_ave_name,CA_mean],axis=1)
    return CA_ave
CA_ave=[aver(i) for i in dataname]


#average price in yr 25/26 for different type of house/and all homes
diff_ave = reduce(lambda left,right: pd.merge(left,right,how="left",on='RegionName'), CA_ave)

,RegionName,sold_15,sold_16,list_15,list_16,one_15,one_16,two_15,two_16,thre_15,...,four_15,four_16,five_15,five_16,cond_15,cond_16,trip_15,trip_16,sfr_15,sfr_16
0,Los Angeles,498737.500000,518508.333333,537666.666667,587328.666667,363468.333333,409424.500000,433116.583333,469545.750000,497107.916667,...,659808.333333,701394.000000,1.038750e+06,1.176233e+06,452570.833333,488478.750000,664705.500000,7.312750e+05,576524.500000,629662.333333
1,Orange,601020.833333,623150.000000,675932.083333,705890.000000,279250.000000,303945.833333,417687.500000,441220.833333,639658.333333,...,841700.000000,883811.583333,1.382648e+06,1.439552e+06,422483.333333,446204.166667,924108.583333,1.120979e+06,787609.750000,820343.541667
2,San Diego,472187.500000,489250.000000,538650.000000,575525.000000,276475.000000,319328.625000,360778.166667,396704.083333,494828.166667,...,684390.666667,716899.958333,1.093432e+06,1.184135e+06,376438.583333,410828.333333,678408.333333,7.657750e+05,609515.250000,641470.583333
3,Riverside,309068.750000,320437.500000,344257.500000,361732.458333,145399.875000,145940.750000,227945.750000,237962.041667,327233.333333,...,394170.750000,413642.833333,4.613150e+05,4.925966e+05,273656.666667,280891.666667,352866.583333,3.872125e+05,356255.250000,376051.708333
4,San Bernardino,264327.083333,272666.666667,271312.500000,291733.333333,NaN,NaN,172779.083333,185468.166667,255624.833333,...,367520.833333,385991.166667,4.659978e+05,4.931395e+05,259899.916667,271735.416667,295638.125000,3.068167e+05,272363.916667,293408.333333


In [196]:
#Features data:Combine dataframe for featuse and sold price in 2015/2016 and save as all_data
file_path = "/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/"
def readdata(file_name):
             #,mergedata=diff_ave):

    dataframe = pd.read_csv(os.path.join(file_path,file_name),header=0)
    
    return(dataframe)

Unemp_frame = readdata("Unemploy_Rate.csv")
Unemp_frame["County"] = [" ".join(i.split(" ")[:-1]) for i in Unemp_frame["County"]]

Pop_frame = readdata("population.csv")
Pop_frame = Pop_frame[["County","Population"]]
Unem_diff=pd.merge(diff_ave,Unemp_frame,how="left",left_on = "RegionName",right_on = "County")
Unem_diff_pop=pd.merge(Unem_diff,Pop_frame,how="left",left_on = "RegionName",right_on = "County")
crim = readdata("crime.csv")

Unme_diff_pop_crim = pd.merge(Unem_diff_pop,crim.iloc[:,1:],how="left", on = "RegionName")
Unem_diff_pop=pd.merge(Unem_diff,Pop_frame,how="left",left_on = "RegionName",right_on = "County")
crim = readdata("crime.csv")

Unme_diff_pop_crim = pd.merge(Unem_diff_pop,crim.iloc[:,1:],how="left", on = "RegionName")
Unme_diff_pop_crim.drop(["County_x",'County_y'],axis=1, inplace=True)

rank = readdata("county_all_rank.csv")
all_rank = pd.merge(Unme_diff_pop_crim,rank,how="left", left_on = "RegionName",right_on = "County")
all_rank.drop(['County'],axis=1, inplace=True)
income = readdata("income.csv")
#income = income.drop(income.iloc[:,:3])
all_income = pd.merge(all_rank,income.iloc[:,3:],how="left", left_on = "RegionName",right_on = "county")
all_income.drop(['county'],axis=1, inplace=True)


CA_sold_t.to_csv("/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/CA_sold_t.csv")
diff_ave.to_csv("/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/diff_ave.csv")
all_income.to_csv("/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/all_data.csv")


all_data = all_income
all_data.head()

,RegionName,sold_15,sold_16,list_15,list_16,one_15,one_16,two_15,two_16,thre_15,...,Highest Rank City 2016,Zip2016,Rank2015,Highest Rank City 2015,Zip2015,Number Students,Number of Ranked Elementary Schools,Number of Ranked Middle Schools,Number of Ranked High Schools,total_income
0,Los Angeles,498737.500000,518508.333333,537666.666667,587328.666667,363468.333333,409424.500000,433116.583333,469545.750000,497107.916667,...,Acton,90017,9.0,Acton,90017,1534702.0,1275.0,502.0,469.0,3263069
1,Orange,601020.833333,623150.000000,675932.083333,705890.000000,279250.000000,303945.833333,417687.500000,441220.833333,639658.333333,...,Anaheim,90620,31.0,Anaheim,90620,496719.0,385.0,113.0,100.0,1009353
2,San Diego,472187.500000,489250.000000,538650.000000,575525.000000,276475.000000,319328.625000,360778.166667,396704.083333,494828.166667,...,Alpine,91901,10.0,Alpine,91901,502352.0,434.0,188.0,145.0,1094157
3,Riverside,309068.750000,320437.500000,344257.500000,361732.458333,145399.875000,145940.750000,227945.750000,237962.041667,327233.333333,...,Banning,92220,123.0,Banning,92220,425225.0,284.0,104.0,105.0,699232
4,San Bernardino,264327.083333,272666.666667,271312.500000,291733.333333,NaN,NaN,172779.083333,185468.166667,255624.833333,...,Adelanto,91701,115.0,Adelanto,91701,409680.0,336.0,125.0,100.0,614325


In [176]:
Pop_frame = readdata("population.csv")
Pop_frame = Pop_frame[["County","Population"]]
Unem_diff=pd.merge(diff_ave,Unemp_frame,how="left",left_on = "RegionName",right_on = "County")
Unem_diff_pop=pd.merge(Unem_diff,Pop_frame,how="left",left_on = "RegionName",right_on = "County")
crim = readdata("crime.csv")
Unme_diff_pop_crim = pd.merge(Unem_diff_pop,crim.iloc[:,1:],how="left", on = "RegionName")
Unem_diff_pop=pd.merge(Unem_diff,Pop_frame,how="left",left_on = "RegionName",right_on = "County")
crim = readdata("crime.csv")
Unme_diff_pop_crim = pd.merge(Unem_diff_pop,crim.iloc[:,1:],how="left", on = "RegionName")
Unme_diff_pop_crim.drop(["County_x",'County_y'],axis=1, inplace=True)
rank = readdata("county_all_rank.csv")
all_rank = pd.merge(Unme_diff_pop_crim,rank,how="left", left_on = "RegionName",right_on = "County")
all_rank.drop(['County'],axis=1, inplace=True)
income = readdata("income.csv")
#income = income.drop(income.iloc[:,:3])
all_income = pd.merge(all_rank,income.iloc[:,3:],how="left", left_on = "RegionName",right_on = "county")
all_income.drop(['county'],axis=1, inplace=True)
CA_sold_t.to_csv("/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/CA_sold_t.csv")
diff_ave.to_csv("/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/diff_ave.csv")
all_income.to_csv("/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/all_data.csv")
all_data = all_income
all_data.head()

In [18]:
####plot multiple time-series line###

def trace(x,y,name):
    trace = Scatter(
            x=x,y=y,
            line=Line(
                width=1
            ),
            name=name
        )
    return trace
traceall = list()

for i in CA_sold_t.keys():
    traceall.append(trace(x=CA_sold_t.index,y=CA_sold_t[i],name=i))


TF = (numpy.identity(len(CA_sold_t.keys()))==1).tolist()
ALLT=[(numpy.repeat(1,len(CA_sold_t.keys()))==1).tolist()]
ALLT.extend(TF)
ALL=['All']
allname=CA_sold_t.keys().tolist()
ALL.extend(allname)


def buttons(TF, label):
    buttons = dict(
        args=['visible',TF],
        label = label,
        method='restyle'
    )
    return buttons

buttonsall = list()
for i in range(len(CA_sold_t.keys())+1):
    buttonsall.append(buttons(TF=ALLT[i],label=ALL[i]))
    
layout = Layout(
    title='Median Sold Price from 1996 to 2016 in Califorina county',
    updatemenus=list([
        dict(
            x=-0.05,
            y=1,
            yanchor='top',
            buttons=buttonsall,
        )
    ]),
)


fig = go.Figure(data=traceall, layout=layout)
py.iplot(fig)

In [200]:
####plot multiple time-series line###

upper_bound0 = go.Scatter(
    name='Upper Bound_med',
    x=CA_sold_t.index,
    y=CA_sold_t.mean(axis=1)+CA_sold_t.std(axis=1),
    mode='lines',
    marker=dict(color="444"),
    line=dict(width=0),
    fillcolor='rgba(68, 68, 68, 0.3)',
    fill='tonexty' )

trace0 = go.Scatter(
    x=CA_sold_t.index,
    y=CA_sold_t.median(axis=1),
    fillcolor='rgba(68, 68, 68, 0.3)',
    name = "median",
    fill='tonexty'
)

trace1 = go.Scatter(
    x=CA_sold_t.index,
    y=CA_sold_t.mean(axis=1),
    line=dict(color='rgb(31, 119, 180)'),
    fillcolor='rgba(68, 68, 68, 0.3)',
    name = "mean",
    fill='tonexty'
)

lower_bound0 = go.Scatter(
    name='Lower Bound_med',
    x=CA_sold_t.index,
    y=CA_sold_t.mean(axis=1)-CA_sold_t.std(axis=1),
    marker=dict(color="444"),
    line=dict(width=0),
    mode='lines' )


data = [lower_bound0,trace0,trace1,upper_bound0]

layout = dict(
    title='Sold Price from 1996 to 2016 in Califorina',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=6,
                     label='6m',
                     step='month',
                     stepmode='backward'),
                dict(count=1,
                    label='1y',
                    step='year',
                    stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(),
        type='date'
    )
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [194]:
###medain sales price compare with US and CA in 2015

median_us=296400
all_data["sold_15"].median()


trace0 = go.Bar(
    x=[median_us, all_data["sold_15"].mean()],
    y=['US ','California'],
    orientation = 'h',
    marker=dict(
        color=['rgba(204,204,204,1)','rgba(222,45,38,0.8)']),
)

data = [trace0]

layout = go.Layout(
    title='Medidan Housing Price between US and California',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='color-bar')

In [197]:
#url = 'http://catalog.civicdashboards.com/dataset/ce409ee1-5128-4b66-aa3e-957dbd4de8ba/resource/6f805645-0836-478d-b168-c1f72d53b4f3/download/5faf934922fd4748a843d55990297d9ftemp.geojson'
california_data = json.load(open('/Users/HUI/Dropbox/16_UCDavis/17W_STA141B/05_Project/california.json'))

### Map Plot

In [95]:
class MultiPolygon(MacroElement):
    """
    !! This is hacked from folium.features.MultiPolyLine !! 
    
    """
    def __init__(self, locations, color=None, weight=None,
                 opacity=None, latlon=True, popup=None):
        super(MultiPolygon, self).__init__()
        self._name = 'MultiPolygon'
        self.data = (_locations_mirror(locations) if not latlon else
                     _locations_tolist(locations))
        self.color = color
        self.weight = weight
        self.opacity = opacity
        if isinstance(popup, text_type) or isinstance(popup, binary_type):
            self.add_children(Popup(popup))
        elif popup is not None:
            self.add_children(popup)

        self._template = Template(u"""
            {% macro script(this, kwargs) %}
                var {{this.get_name()}} = L.multiPolygon(
                    {{this.data}},
                    {
                        {% if this.color != None %}color: '{{ this.color }}',{% endif %}
                        {% if this.weight != None %}weight: {{ this.weight }},{% endif %}
                        {% if this.opacity != None %}opacity: {{ this.opacity }},{% endif %}
                        });
                {{this._parent.get_name()}}.addLayer({{this.get_name()}});
            {% endmacro %}
            """)  # noqa

    def _get_self_bounds(self):
        """Computes the bounds of the object itself (not including it's children)
        in the form [[lat_min, lon_min], [lat_max, lon_max]]
        """
        bounds = [[None, None], [None, None]]
        for point in iter_points(self.data):
            bounds = [
                [
                    none_min(bounds[0][0], point[0]),
                    none_min(bounds[0][1], point[1]),
                ],
                [
                    none_max(bounds[1][0], point[0]),
                    none_max(bounds[1][1], point[1]),
                ],
            ]
        return bounds

In [96]:
geo_dict = {}
for x in range(len(california_data['features'])):
    # I ignore the last eleven characters in the name since the geojson file includes ' County, CO' in the county names and the population data does not
    name = california_data['features'][x]['properties']['name'][:-11]
    if name in all_data['RegionName'].unique():
        geo_dict[name] = california_data['features'][x]
    else:
        print 'not in: ', name

ser = pd.Series(geo_dict.values(), index = geo_dict.keys())
ser.name = 'coordinates'
all_data_cord = all_data.join(ser, on='RegionName')

income_rate = [round(i,6) for i in all_data["total_income"]/all_data["Population"]]
crim_rate = [round(i,6) for i in all_data["CrimeCases"]/all_data["Population"]]

all_data_sub = all_data_cord[['RegionName','sold_15','sold_16','list_15','list_16','Score2016','Score2015','Unemploy_Rate','Population',"coordinates"]]
all_data_sub['income_rate'] = income_rate
all_data_sub['crim_rate'] = crim_rate

/Users/HUI/miniconda2/lib/python2.7/site-packages/ipykernel/__main__.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/HUI/miniconda2/lib/python2.7/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [97]:
colors = ['#ffffe0','#fffddb','#fffad7','#fff7d1','#fff5cd','#fff2c8',
          '#fff0c4','#ffedbf','#ffebba','#ffe9b7','#ffe5b2','#ffe3af',
          '#ffe0ab','#ffdda7','#ffdba4','#ffd9a0','#ffd69c','#ffd399',
          '#ffd196','#ffcd93','#ffca90','#ffc88d','#ffc58a','#ffc288',
          '#ffbf86','#ffbd83','#ffb981','#ffb67f','#ffb47d','#ffb17b',
          '#ffad79','#ffaa77','#ffa775','#ffa474','#ffa172','#ff9e70',
          '#ff9b6f','#ff986e','#ff956c','#fe916b','#fe8f6a','#fd8b69',
          '#fc8868','#fb8567','#fa8266','#f98065','#f87d64','#f77a63',
          '#f67862','#f57562','#f37261','#f37060','#f16c5f','#f0695e',
          '#ee665d','#ed645c','#ec615b','#ea5e5b','#e85b59','#e75859',
          '#e55658','#e45356','#e35056','#e14d54','#df4a53','#dd4852',
          '#db4551','#d9434f','#d8404e','#d53d4d','#d43b4b','#d2384a',
          '#cf3548','#cd3346','#cc3045','#ca2e43','#c72b42','#c52940',
          '#c2263d','#c0233c','#be213a','#bb1e37','#ba1c35','#b71933',
          '#b41731','#b2152e','#b0122c','#ac1029','#aa0e27','#a70b24',
          '#a40921','#a2071f','#a0051c','#9d0419','#990215','#970212',
          '#94010e','#91000a','#8e0006','#8b0000', '#8b0000']

scl = dict(zip(range(0, 101), colors))
colorscl = [[i * .01, v] for i,v in enumerate(colors)]

In [98]:
def get_scl(obj,number):
    """
    assign color
    """
    if pd.isnull(obj):
        return scl[0] 
    else:
        frac = int(obj/number)
        return scl[frac]

all_data_sub['color'] = all_data_sub['sold_15'].apply(get_scl,number = 12500)


/Users/HUI/miniconda2/lib/python2.7/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [99]:
requests_cache.install_cache("cache") 
url = requests.get("https://en.wikipedia.org/wiki/User:Michael_J/County_table") 
soup = BeautifulSoup(url.text)

sort = []
state = []
FIPS = []
county = []
county_seat = []
population = []
land_area_km = []
land_area_ml = []
water_area_km = []
water_area_ml = []
total_area_km = []
total_area_ml = []
lat = []
lon = []
 
table = soup.find("table", { "class" : "wikitable sortable" })
 
for row in table.findAll("tr"):
    cells = row.findAll("td")
    #For each "tr", assign each "td" to a variable.
    if len(cells) == 14:
        sort.append(cells[0].find(text=True))
        state.append(cells[1].find(text=True))
        FIPS.append(cells[2].find(text=True))
        county.append(cells[3].find(text=True))
        county_seat.append(cells[4].find(text=True))
        population.append(cells[5].find(text=True))
        land_area_km.append(cells[6].find(text=True))
        land_area_ml.append(cells[7].find(text=True))
        water_area_km.append(cells[8].find(text=True))
        water_area_ml.append(cells[9].find(text=True))
        total_area_km.append(cells[10].find(text=True))
        total_area_ml.append(cells[11].find(text=True))
        lat.append(cells[12].find(text=True))
        lon.append(cells[13].find(text=True))

cord = [('county', county[186:244]),
         ('lat', lat[186:244]),
         ('lon', lon[186:244]),
         ]
center_cord = pd.DataFrame.from_items(cord)

all_data_sub_cen = pd.merge(all_data_sub, center_cord, left_on='RegionName', right_on='county')
all_data_sub_cen["lon"] = ['-'+i[1:9] for i in all_data_sub_cen["lon"]]
all_data_sub_cen["lat"] = [i[1:8] for i in all_data_sub_cen["lat"]]


In [100]:
mapbox_access_token = 'pk.eyJ1IjoiZGFuaWVsbGU5MTUxNSIsImEiOiJjaXpyeW44cTYwMGtvMzJsOXZtNTEwNmJtIn0.8d-m0sZPWnlRzH-T4OlAiA'

In [101]:
layers_ls = []
for x in all_data_sub.index:
    item_dict = dict(sourcetype = 'geojson',
                     source = all_data_sub.ix[x]['coordinates'],
                     type = 'fill',
                     color = all_data_sub.ix[x]['color'])
    layers_ls.append(item_dict)
test = []
for i in range(len(all_data_sub_cen["RegionName"].tolist())):
    test.append(all_data_sub_cen["RegionName"].tolist()[i] + '</br>' + 
                'Sold Price: ' +str(all_data_sub_cen["sold_15"].tolist()[i])+ '</br>' + 
                'List Price: ' +str(all_data_sub_cen["list_15"].tolist()[i])+ '</br>' + 
                'School Score: '+str(all_data_sub_cen["Score2015"].tolist()[i])+ '</br>' +
                'Population:' +str(all_data_sub_cen["Population"].tolist()[i]) +'</br>' +
                'Ave Income: '+str(all_data_sub_cen["income_rate"].tolist()[i]) +'</br>' +
                'Criminal Rate: '+str(all_data_sub_cen["crim_rate"].tolist()[i])+'</br>' +
                'Unemploy Rate: '+str(all_data_sub_cen["Unemploy_Rate"].tolist()[i]))

In [102]:
data = go.Data([
        go.Scattermapbox(
            lat=all_data_sub_cen["lat"].tolist(),
            lon=all_data_sub_cen["lon"].tolist(),
            text=test,
            mode = 'markers',
                    marker = go.Marker(cmax=int(all_data_sub['sold_15'].max()/0.01),cmin=0,colorscale = colorscl,
                        showscale = True,autocolorscale=False,color=range(0,100),
                    colorbar= go.ColorBar(len = .89)
                                       ),
        )
    ])


layout = go.Layout(
    height=600,
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        layers=layers_ls,
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=38.039139,
            lon=-120.803628
        ),
        pitch=0,
        zoom=5,
        style='light'
               )
)
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='county-level-choropleths-python')

In [121]:
#plot scatter matrix
crim_rate = ({"crimerate":all_data["CrimeCases"]/all_data["Population"]})
crim_rate_log = ({"logcrimerate":[math.log(i) for i in all_data["CrimeCases"]/all_data["Population"]]})
income_rate = ({"incomerate":all_data["total_income"]/all_data["Population"]})
population_log =  ({"logpopulation":[math.log(i) for i in all_data["Population"]]})
dataframe = all_data[['sold_15','Unemploy_Rate','Score2015']]

all_data_sub_2 = pd.concat([dataframe,pd.DataFrame(crim_rate_log),
                          pd.DataFrame(population_log),pd.DataFrame(income_rate)],axis=1)

fig = ff.create_scatterplotmatrix(all_data_sub_2, diag='histogram',
                                  colormap='Blues', height=800, width=800)
py.iplot(fig, filename = 'Scatterplot Matrix')

In [188]:
###Recent 5 year growth rate in county
###Calcuat CAGR Compound Annual Growth Rate
CAGR= ({'CAGR':((CA_sold_t['2016'].mean(axis=0)/CA_sold_t['2012'].mean(axis=0))**0.2-1)*100})
CAGR = pd.DataFrame(CAGR)
CAGR['RegionName']=CAGR.index

ser.name = 'coordinates'
CAGR = CAGR.join(ser, on='RegionName')
###Give negative as 0####
CAGR.set_value('Del Norte', "CAGR", 0)
#####
CAGR['color'] = CAGR['CAGR'].apply(get_scl,number = 0.15)

CAGR_cen = pd.merge(CAGR, center_cord, left_on='RegionName', right_on='county')
CAGR_cen["lon"] = ['-'+i[1:9] for i in CAGR_cen["lon"]]
CAGR_cen["lat"] = [i[1:8] for i in CAGR_cen["lat"]]
colorscl = [[i * .01, v] for i,v in enumerate(colors)]


layers_ls = []
for x in CAGR.index:
    item_dict = dict(sourcetype = 'geojson',
                     source = CAGR.ix[x]['coordinates'],
                     type = 'fill',
                     color = CAGR.ix[x]['color'])
    layers_ls.append(item_dict)
test = []
for i in range(len(CAGR_cen["RegionName"].tolist())):
    test.append(CAGR_cen["RegionName"].tolist()[i] + '</br>' + 
                'CAGR: ' +str(CAGR_cen["CAGR"].tolist()[i]) 
                )

In [189]:
data = go.Data([
        go.Scattermapbox(
            lat=CAGR_cen["lat"].tolist(),
            lon=CAGR_cen["lon"].tolist(),
            text=test,
            mode = 'markers',
                    marker = go.Marker(cmax=int(CAGR['CAGR'].max()/10),cmin=0,colorscale = colorscl,
                        showscale = True,autocolorscale=False,color=range(0,10),
                    colorbar= go.ColorBar(len = .89)
                                       ),
        )
    ])


layout = go.Layout(
    height=600,
    autosize=True,
    hovermode='closest',
    mapbox=dict(
        layers=layers_ls,
        accesstoken=mapbox_access_token,
        bearing=0,
        center=dict(
            lat=38.039139,
            lon=-120.803628
        ),
        pitch=0,
        zoom=5,
        style='light'
               )
)

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='CAGR')

In [201]:
CAGR.sort_values(by="CAGR",ascending=False).head()

,CAGR,RegionName,coordinates,color
Colusa,13.231003,Colusa,"{u'geometry': {u'type': u'MultiPolygon', u'coo...",#aa0e27
San Benito,11.876452,San Benito,"{u'geometry': {u'type': u'MultiPolygon', u'coo...",#c0233c
Stanislaus,11.828371,Stanislaus,"{u'geometry': {u'type': u'MultiPolygon', u'coo...",#c2263d
Solano,11.604935,Solano,"{u'geometry': {u'type': u'MultiPolygon', u'coo...",#c52940
San Joaquin,11.310486,San Joaquin,"{u'geometry': {u'type': u'MultiPolygon', u'coo...",#ca2e43


In [203]:
all_data.sort_values(by='sold_16',ascending=False)

,RegionName,sold_15,sold_16,list_15,list_16,one_15,one_16,two_15,two_16,thre_15,...,Highest Rank City 2016,Zip2016,Rank2015,Highest Rank City 2015,Zip2015,Number Students,Number of Ranked Elementary Schools,Number of Ranked Middle Schools,Number of Ranked High Schools,total_income
11,San Francisco,1.107921e+06,1.173833e+06,1.021083e+06,1.123579e+06,778854.166667,812199.000000,1.046000e+06,1.164208e+06,1.331000e+06,...,San Francisco,94102,331.0,San Francisco,94102,58592.0,75.0,25.0,22.0,353287
13,San Mateo,9.192500e+05,9.963750e+05,1.002279e+06,1.119322e+06,452680.833333,515988.583333,7.511732e+05,8.225230e+05,9.820146e+05,...,Atherton,94002,1.0,Atherton,94002,94986.0,106.0,49.0,28.0,259711
25,Marin,8.582417e+05,9.094333e+05,1.018950e+06,1.138625e+06,NaN,NaN,6.784375e+05,7.144562e+05,9.244036e+05,...,Bolinas,94903,12.0,Bolinas,94903,33029.0,38.0,17.0,13.0,103670
5,Santa Clara,7.992500e+05,8.563750e+05,8.225505e+05,8.882302e+05,441718.666667,467069.833333,5.505682e+05,6.242908e+05,7.652370e+05,...,Campbell,94024,4.0,Campbell,94024,275810.0,248.0,93.0,75.0,621463
6,Alameda,6.147958e+05,6.618083e+05,6.279979e+05,6.877866e+05,350766.666667,398109.916667,4.469397e+05,5.095338e+05,6.270787e+05,...,Alameda,94501,44.0,Alameda,94501,224453.0,219.0,87.0,83.0,558907
23,Santa Cruz,6.257829e+05,6.607250e+05,6.888250e+05,7.400312e+05,443450.000000,473708.291667,5.034117e+05,5.393707e+05,7.902271e+05,...,Ben Lomond,95005,30.0,Ben Lomond,95005,41063.0,41.0,23.0,20.0,94802
1,Orange,6.010208e+05,6.231500e+05,6.759321e+05,7.058900e+05,279250.000000,303945.833333,4.176875e+05,4.412208e+05,6.396583e+05,...,Anaheim,90620,31.0,Anaheim,90620,496719.0,385.0,113.0,100.0,1009353
33,Napa,5.265542e+05,5.472333e+05,6.538833e+05,6.817292e+05,NaN,NaN,5.179662e+05,5.344562e+05,6.577041e+05,...,Angwin,94508,144.0,Angwin,94508,20978.0,22.0,9.0,8.0,49494
10,Ventura,5.022188e+05,5.242958e+05,5.899828e+05,6.127532e+05,285379.166667,296204.166667,3.690928e+05,3.940768e+05,5.139000e+05,...,Camarillo,91362,118.0,Camarillo,91362,141762.0,130.0,49.0,42.0,268969
0,Los Angeles,4.987375e+05,5.185083e+05,5.376667e+05,5.873287e+05,363468.333333,409424.500000,4.331166e+05,4.695458e+05,4.971079e+05,...,Acton,90017,9.0,Acton,90017,1534702.0,1275.0,502.0,469.0,3263069
